In [1]:
from attr import evolve
import pandas as pd
from floweaver import *

In [2]:
dataset = Dataset.from_csv("../docs/cookbook/us-energy-consumption.csv", dim_process_filename="../docs/cookbook/us-energy-consumption-processes.csv")

In [3]:
sources = ['Solar', 'Nuclear', 'Hydro', 'Wind', 'Geothermal',
           'Natural_Gas', 'Coal', 'Biomass', 'Petroleum']

uses = ['Residential', 'Commercial', 'Industrial', 'Transportation']

In [4]:
nodes = {
    'sources': ProcessGroup('type == "source"', Partition.Simple('process', sources), title='Sources'),
    'imports': ProcessGroup(['Net_Electricity_Import'], title='Net electricity imports'),
    'electricity': ProcessGroup(['Electricity_Generation'], title='Electricity Generation'),
    'uses': ProcessGroup('type == "use"', partition=Partition.Simple('process', uses)),
    
    'energy_services': ProcessGroup(['Energy_Services'], title='Energy services'),
    'rejected': ProcessGroup(['Rejected_Energy'], title='Rejected energy'),
    
    'direct_use': Waypoint(Partition.Simple('source', [
        # This is a hack to hide the labels of the partition, there should be a better way...
        (' '*i, [k]) for i, k in enumerate(sources)
    ])),
}

ordering = [
    [[], ['sources'], []],
    [['imports'], ['electricity', 'direct_use'], []],
    [[], ['uses'], []],
    [[], ['rejected', 'energy_services'], []]
]

bundles = [
    Bundle('sources', 'electricity'),
    Bundle('sources', 'uses', waypoints=['direct_use']),
    Bundle('electricity', 'uses'),
    Bundle('imports', 'uses'),
    Bundle('uses', 'energy_services'),
    Bundle('uses', 'rejected'),
    Bundle('electricity', 'rejected'),
]

In [5]:
palette = {
    'Solar': 'gold',
    'Nuclear': 'red',
    'Hydro': 'blue',
    'Wind': 'purple',
    'Geothermal': 'brown',
    'Natural_Gas': 'steelblue',
    'Coal': 'black',
    'Biomass': 'lightgreen',
    'Petroleum': 'green',
    'Electricity': 'orange',
    'Rejected energy': 'lightgrey',
    'Energy services': 'dimgrey',
}

In [6]:
sdd = SankeyDefinition(nodes, bundles, ordering,
                       flow_partition=dataset.partition('type'))
sankey_data = weave(sdd, dataset, palette=palette)

sankey_data.to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

In [7]:
model = model_inputs(sankey_data, group_nodes = True)

In [8]:
opt_order = optimise_node_order(model, group_nodes = True)

ugabuga1
ugabuga1.25
ugabuga1.5
ugabuga2
ugabuga3
<generator object optimise_node_order.<locals>.<genexpr> at 0x00000203FB3E3D10>
ugabuga4
ugabuga5
ugabuga6
ugabuga7
ugabuga8


In [9]:
sankey_data_evolved = evolve(sankey_data, ordering = opt_order)
sankey_data_evolved.to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

In [10]:
s_model = straightness_model(sankey_data)

In [11]:
ys = optimise_position(s_model, wslb = 5)

In [12]:
sankey_data.to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True, forceY = ys, y_scale = 3)
